https://www.youtube.com/watch?v=Q2UoBmvTV78

![alt text](ss.png "Title")

In [1]:
!ls -ltr

total 288
-rw-r--r-- 1 atif atif     56 Nov 14 17:19  multi_deli_file.txt
drwxr-xr-x 2 atif atif   4096 Nov 14 18:09  data2
drwxr-xr-x 2 atif atif   4096 Nov 14 18:10  data1
drwxr-xr-x 2 atif atif   4096 Nov 14 18:14  data3
-rw-r--r-- 1 atif atif   7729 Nov 14 18:22  multi_delimeted_file_handling.ipynb
-rw-r--r-- 1 atif atif   9716 Nov 14 18:23 'Load Data from multiple directory.ipynb'
-rw-r--r-- 1 atif atif 249704 Nov 14 18:55  ss.png
-rw-r--r-- 1 atif atif    793 Nov 14 18:58 'Explode and Posexplode.ipynb'
-rw-r--r-- 1 atif atif     89 Nov 14 19:02  explode_posexplode_example.txt


In [2]:
!cat explode_posexplode_example.txt

Name|Age|Education
Azar|25|MBA,BE,HSC
HARI|32|
KUMAR|35|ME,BE,Diploma
IMAM|24|B.Tech,HSC


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('explode_posexplode').getOrCreate()

In [5]:
df_in =spark.read.csv('explode_posexplode_example.txt',sep='|',header=True)

In [6]:
df_in.show()

+-----+---+-------------+
| Name|Age|    Education|
+-----+---+-------------+
| Azar| 25|   MBA,BE,HSC|
| HARI| 32|         null|
|KUMAR| 35|ME,BE,Diploma|
| IMAM| 24|   B.Tech,HSC|
+-----+---+-------------+



In [7]:
from pyspark.sql.functions import explode,split

In [10]:
df_explode =df_in.withColumn("Degrees",explode(split("Education",",")))

In [11]:
df_explode.show()

+-----+---+-------------+-------+
| Name|Age|    Education|Degrees|
+-----+---+-------------+-------+
| Azar| 25|   MBA,BE,HSC|    MBA|
| Azar| 25|   MBA,BE,HSC|     BE|
| Azar| 25|   MBA,BE,HSC|    HSC|
|KUMAR| 35|ME,BE,Diploma|     ME|
|KUMAR| 35|ME,BE,Diploma|     BE|
|KUMAR| 35|ME,BE,Diploma|Diploma|
| IMAM| 24|   B.Tech,HSC| B.Tech|
| IMAM| 24|   B.Tech,HSC|    HSC|
+-----+---+-------------+-------+



### but as you can we have missed `HARI` as it didn't had any entry in Education column

#### we will use explode_out instead of explode

In [12]:
from pyspark.sql.functions import explode_outer,split

In [15]:
df_explode_outer =df_in.withColumn("Degrees",explode_outer(split("Education",",")))

In [16]:
df_explode_outer.show()

+-----+---+-------------+-------+
| Name|Age|    Education|Degrees|
+-----+---+-------------+-------+
| Azar| 25|   MBA,BE,HSC|    MBA|
| Azar| 25|   MBA,BE,HSC|     BE|
| Azar| 25|   MBA,BE,HSC|    HSC|
| HARI| 32|         null|   null|
|KUMAR| 35|ME,BE,Diploma|     ME|
|KUMAR| 35|ME,BE,Diploma|     BE|
|KUMAR| 35|ME,BE,Diploma|Diploma|
| IMAM| 24|   B.Tech,HSC| B.Tech|
| IMAM| 24|   B.Tech,HSC|    HSC|
+-----+---+-------------+-------+



### Now we need positional index of degrees


#### So, we will need posexplode , but posexplode doesn't combine with withColumn

In [18]:
from pyspark.sql.functions import posexplode_outer,split

In [21]:
df_pos_explode_outer =df_in.withColumn("Degrees",posexplode_outer(split("Education",",")))

AnalysisException: The number of aliases supplied in the AS clause does not match the number of columns output by the UDTF expected 2 aliases but got Degrees ;

In [35]:
df_pos_explode_outer =df_in.select("*",posexplode_outer(split("Education",",")))

In [36]:
df_pos_explode_outer.show()

+-----+---+-------------+----+-------+
| Name|Age|    Education| pos|    col|
+-----+---+-------------+----+-------+
| Azar| 25|   MBA,BE,HSC|   0|    MBA|
| Azar| 25|   MBA,BE,HSC|   1|     BE|
| Azar| 25|   MBA,BE,HSC|   2|    HSC|
| HARI| 32|         null|null|   null|
|KUMAR| 35|ME,BE,Diploma|   0|     ME|
|KUMAR| 35|ME,BE,Diploma|   1|     BE|
|KUMAR| 35|ME,BE,Diploma|   2|Diploma|
| IMAM| 24|   B.Tech,HSC|   0| B.Tech|
| IMAM| 24|   B.Tech,HSC|   1|    HSC|
+-----+---+-------------+----+-------+



In [37]:
# Now lets rename column 'col' and pos to index

In [38]:
df_pos_explode_outer =df_pos_explode_outer.withColumnRenamed("col",'Qualification').\
withColumnRenamed("pos","Index")

In [39]:
df_pos_explode_outer.show()

+-----+---+-------------+-----+-------------+
| Name|Age|    Education|Index|Qualification|
+-----+---+-------------+-----+-------------+
| Azar| 25|   MBA,BE,HSC|    0|          MBA|
| Azar| 25|   MBA,BE,HSC|    1|           BE|
| Azar| 25|   MBA,BE,HSC|    2|          HSC|
| HARI| 32|         null| null|         null|
|KUMAR| 35|ME,BE,Diploma|    0|           ME|
|KUMAR| 35|ME,BE,Diploma|    1|           BE|
|KUMAR| 35|ME,BE,Diploma|    2|      Diploma|
| IMAM| 24|   B.Tech,HSC|    0|       B.Tech|
| IMAM| 24|   B.Tech,HSC|    1|          HSC|
+-----+---+-------------+-----+-------------+



In [40]:
# Now lets drop the column Education

In [41]:
df_pos_explode_outer = df_pos_explode_outer.drop("Education")

In [42]:
df_pos_explode_outer.show()

+-----+---+-----+-------------+
| Name|Age|Index|Qualification|
+-----+---+-----+-------------+
| Azar| 25|    0|          MBA|
| Azar| 25|    1|           BE|
| Azar| 25|    2|          HSC|
| HARI| 32| null|         null|
|KUMAR| 35|    0|           ME|
|KUMAR| 35|    1|           BE|
|KUMAR| 35|    2|      Diploma|
| IMAM| 24|    0|       B.Tech|
| IMAM| 24|    1|          HSC|
+-----+---+-----+-------------+

